# Precission 2

In [1]:
import os
import socket
import subprocess
import pandas as pd
import io
from contextlib import redirect_stdout
import itertools
import warnings
from datetime import datetime
import time
import timeout_decorator
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def islocal(): return socket.gethostname()=="rc-idsia-macBook.local"
def isserver(): return socket.gethostname()=="758606aabdd8"
def strdate(): return datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
print(socket.gethostname())

if islocal():
    prj_path = "/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/"
    os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home"
elif isserver():
    prj_path="/home/rcabanas/crema/CreMA/"
else:
    warnings.warn("Unknown server, set prj_path variable manually.")
os.chdir(prj_path)
!pwd

rc-idsia-macBook.local
/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici


In [13]:
exp_folder = f"{prj_path}/papers/pgm20/experiments/"
log_folder = f"{exp_folder}/logs/"
res_folder = f"{exp_folder}/results/"
jarfile=f"{prj_path}/target/credici-0.1.1-jar-with-dependencies.jar"

rebuild = True

In [14]:
if rebuild:
    !mvn clean compile assembly:single

[INFO] Scanning for projects...
[INFO] 
[INFO] --------------------------< ch.idsia:credici >--------------------------
[INFO] Building credici 0.1.1
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:2.5:clean (default-clean) @ credici ---
[INFO] Deleting /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/target
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ credici ---
[WARNING] Using platform encoding (US-ASCII actually) to copy filtered resources, i.e. build is platform dependent!
[INFO] Copying 0 resource
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ credici ---
[INFO] Changes detected - recompiling the module!
[WARNING] File encoding has not been set, using platform encoding US-ASCII, i.e. build is platform dependent!
[INFO] Compiling 25 source files to /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/target/classes
[WARNING] /Users/rcabanas/Goo

In [15]:
disable_java = False
@timeout_decorator.timeout(7*1*60)
def run_java(jarfile, javafile):
    cmd=f"java -cp {jarfile} {javafile}"
    if disable_java: cmd= "echo 0,0,0,0,0,0"    
    print(cmd)
    result =subprocess.run(cmd, universal_newlines = True, shell=True,stdout = subprocess.PIPE)
    print(result.stdout)
    return result

# ChainNonMarkovian 6 5 1 -1 0 CCALP 1234
def run_chain(model, N, endovarsize, exovarsize, target, obsvar, dovar, method, seed):
    
    
    print(strdate())
    exovarsize = exovarsize or endovarsize*endovarsize + 1
    
    if obsvar is None: obsvar = -1;
    elif obsvar<0: obsvar = N + obsvar;
    
    if dovar is None: dovar = -1;    
    elif dovar<0: dovar = N + dovar;
    
    if target is None: target = N//2;
    elif target<0: target = N + target;
    
    warmups = 0
    repetitions = 1
    eps = 0.0001
    #ChainNonMarkovian 4  -v 5 -V 9 -t 1 -o -1 -d 0 -m  CCALP --seed 12234 --warmpus 0 --repetitions 1
    javafile = f"{exp_folder}/RunExperiments.java {model} {N} -v {endovarsize} -V {exovarsize} -t {target} -o {obsvar} -d {dovar} -m {method} -e {eps} -s {seed} -w {warmups} -r {repetitions} "  
    try:
        result = run_java(jarfile, javafile)
        output = [float(x) for x in result.stdout.splitlines()[-1].split(",")]
    except:
        output = [float("inf"),float("inf")]+[float("nan")]* (endovarsize*2)
        
    return output

def run_chain_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("ChainMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_chain_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("ChainNonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)


def run_hmm_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("HMM-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_hmm_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("HMM-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)



def run_rhmm_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("RHMM-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_rhmm_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("RHMM-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_squares_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("Squares-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_squares_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("Squares-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)



def run_chain_terbin_markovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("TerBinChainMarkovian", N, 2, None, target, obsvar, dovar, method, seed)

def run_chain_terbin_nonmarkovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("TerBinChainNonMarkovian", N, 2, None, target, obsvar, dovar, method, seed)



In [16]:

def run_experiments(f, args, outkeys, fargs=None, verbose=False, lenght_dep_vars = None, non_evaluable=[]):
    
    
    
    print("=========")
    print(args)
    print("=========")         
        
    result = pd.DataFrame(columns=list(args.keys())+list(outkeys))
    log_file = f"{log_folder}{strdate()}_{f.__name__}.txt"
    
    data = pd.DataFrame(list(itertools.product(*list(args.values()))), columns = args.keys())
    
    fargs = fargs or {}
    for k,v in fargs.items():
        data[k]=data.apply(v, axis=1)
        
        
    non_evaluable = non_evaluable or [] 
    
    lenght_dep_vars = lenght_dep_vars or ["N"]
    
    def is_evaluable(args):
        current = {k:v for (k,v) in args.items() if k not in lenght_dep_vars}
        previous = [{k:v for (k,v) in a.items() if k not in lenght_dep_vars} for a in non_evaluable]
        
        print(f"current: {current}")
        print(f"previous: {previous}")

        
        return not current in previous
       
    def single_experiment(argsv):
        if is_evaluable(argsv):
            outvals = f(**argsv)
            if np.isnan(outvals).any(): 
                non_evaluable.append(argsv)
                print(f"setting as not evaluable: {argsv}")
        else:
            outvals=[float("nan")]*len(outkeys)
        return outvals    
        
    
    for argsv in data.to_dict(orient="row"):    
        
        strio = io.StringIO()
        
           
        print(strdate())
        with open(log_file, 'a+') as logger:
            if verbose == False:
                with redirect_stdout(strio):
                    print(strdate())
                    outvals = single_experiment(argsv)
            else:
                outvals = single_experiment(argsv)   
            
            logger.write(strio.getvalue())
            
            
        result = result.append({**argsv, **dict(zip(list(outkeys), outvals))}, ignore_index=True)
        print(dict(result.iloc[-1]))
        print("\n\n")
    
    return result

def get_args(**kwargs): return kwargs

##




In [17]:
SEED = [1234+i*1234//2 for i in range(1,100,1)]
res = {}
len(SEED)

99

Note that all the cells below are disabled, just change the `if` condition in the desiered block of experiments 

In [18]:
# with evidence HMM nonmarkovian 
if True:
    endovarsize = 2
    args = dict(N=range(3,11), dovar=[0], seed=SEED, exovarsize=[6], method=["CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 1))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_hmm_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_hmm_nonmarkovian_prec.csv"
    results1.to_csv(res_file)

    res["hmm_nonmarkovian_prec"] = results1

{'N': range(3, 11), 'dovar': [0], 'seed': [1851, 2468, 3085, 3702, 4319, 4936, 5553, 6170, 6787, 7404, 8021, 8638, 9255, 9872, 10489, 11106, 11723, 12340, 12957, 13574, 14191, 14808, 15425, 16042, 16659, 17276, 17893, 18510, 19127, 19744, 20361, 20978, 21595, 22212, 22829, 23446, 24063, 24680, 25297, 25914, 26531, 27148, 27765, 28382, 28999, 29616, 30233, 30850, 31467, 32084, 32701, 33318, 33935, 34552, 35169, 35786, 36403, 37020, 37637, 38254, 38871, 39488, 40105, 40722, 41339, 41956, 42573, 43190, 43807, 44424, 45041, 45658, 46275, 46892, 47509, 48126, 48743, 49360, 49977, 50594, 51211, 51828, 52445, 53062, 53679, 54296, 54913, 55530, 56147, 56764, 57381, 57998, 58615, 59232, 59849, 60466, 61083, 61700, 62317], 'exovarsize': [6], 'method': ['CCVE', 'CCALPeps']}
2020-07-08-17-05-48
{'N': 3, 'dovar': 0, 'seed': 1851, 'exovarsize': 6, 'method': 'CCVE', 'time': 1188.672, 'query_time': 399.08, 'lowerbound0': 0.019950828514444123, 'upperbound0': 0.9498805557301817, 'lowerbound1': 0.0501194

In [ ]:
# with evidence RevHMM nonmarkovian
if False:
    endovarsize=2
    args = dict(N=[3,4,5,6], dovar=[0], seed=SEED, exovarsize=[6], method=["CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 2))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_rhmm_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_rhmm_nonmarkovian_prec.csv"
    results1.to_csv(res_file)

    res["rhmm_nonmarkovian_prec"] = results1

In [ ]:
# with evidence Squares nonmarkovian 
if False:
    endovarsize = 2
    args = dict(N=list(range(3,11)), dovar=[0], seed=SEED, exovarsize=[6], method=["CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 1))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_squares_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_squares_nonmarkovian_prec.csv"
    results1.to_csv(res_file)

    res["squares_nonmarkovian_prec"] = results1
